In [1]:
import librosa
import csv
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv('train.csv',on_bad_lines='skip')
train=train.dropna()
check=pd.read_csv("check.csv",on_bad_lines='skip')
check=check.dropna()

In [3]:
train.head()

,label,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data47095,data47096,data47097,data47098,data47099,data47100,data47101,data47102,data47103,data47104
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000610,0.000854,0.000549,0.001007,0.000854,0.000610,0.000885,0.000519,0.001251,0.000427
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000092,0.000092,-0.000122,-0.000336,-0.000122,-0.000397,-0.000061,-0.000549,0.000580,-0.000305
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001404,0.001495,0.001312,0.000610,0.001495,0.001343,0.000824,0.001160,0.000488,0.001160
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002930,0.001923,0.002472,0.002258,0.002045,0.002228,0.001190,0.001831,0.001617,0.001434
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003021,0.002869,0.002869,0.002960,0.002747,0.003204,0.003052,0.003418,0.003418,0.003052


In [4]:
check.head()

,label,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data47095,data47096,data47097,data47098,data47099,data47100,data47101,data47102,data47103,data47104
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000824,-0.000580,-0.001434,-0.001343,-0.001068,-0.001404,-0.001678,-0.001709,-0.001617,-0.001984
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001495,0.000946,0.000610,0.001373,0.000977,0.001648,0.001892,0.002136,0.002472,0.002380
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000671,0.000244,0.000305,0.000061,0.000031,0.000519,-0.000122,0.001038,0.000336,0.000244
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000397,-0.001434,-0.000916,-0.001038,-0.000275,-0.000946,-0.001099,-0.000580,-0.000916,-0.000244
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000885,0.001892,0.001404,0.001007,0.001495,0.001312,0.000885,0.001068,0.000641,0.000763


In [5]:
#corner=1 wall=0
train_label=train['label']
check_label=check['label']
del train['label']
del check['label']

In [6]:
train_label.shape

(1379,)

In [7]:
check_label.shape

(420,)

In [8]:
train.shape

(1379, 47104)

In [9]:
check.shape

(420, 47104)

In [10]:
x_train=np.zeros((np.size(train,0),47104))
tr=np.array(train)
ch=np.array(check)
for i in range(0,np.size(tr,0)):
    window=librosa.filters.get_window(tr[i], Nx=47104)
    x_train[i,:]=librosa.filters.window_sumsquare(window,n_frames=1,hop_length=4000,n_fft=47104)
x_check=np.zeros((np.size(check,0),47104))
for i in range(0,np.size(ch,0)):
    window2=librosa.filters.get_window(ch[i], Nx=47104)
    x_check[i,:]=librosa.filters.window_sumsquare(window2,n_frames=1,hop_length=4000,n_fft=47104)

In [11]:
x_train.shape

(1379, 47104)

In [12]:
osm=(x_train-x_train.min())/(x_train.max()-x_train.min())

In [13]:
wow=(x_check-x_check.min())/(x_check.max()-x_check.min())

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Dropout,
    BatchNormalization,
    Embedding,
    LSTM
)

model = Sequential()
model.add(Dense(units=100,activation='softmax',input_shape=(47104,)))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(units=50,activation='linear',input_shape=(47104,)))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(units=1, activation="sigmoid"))

In [47]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               4710500   
                                                                 
 batch_normalization_6 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 50)                5050      
                                                                 
 batch_normalization_7 (Batc  (None, 50)               200       
 hNormalization)                                                 
                                                                 
 dropout_7 (Dropout)         (None, 50)               

In [52]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adamax')

In [53]:
model.fit(osm,train_label,epochs=20,verbose=1,validation_data=(wow,check_label))

Epoch 1/20
44/44 [==============================] - 2s 27ms/step - loss: 0.0000e+00 - accuracy: 0.9362 - val_loss: 0.0000e+00 - val_accuracy: 0.6690
Epoch 2/20
44/44 [==============================] - 1s 24ms/step - loss: 0.0000e+00 - accuracy: 0.9507 - val_loss: 0.0000e+00 - val_accuracy: 0.6548
Epoch 3/20
44/44 [==============================] - 1s 23ms/step - loss: 0.0000e+00 - accuracy: 0.9413 - val_loss: 0.0000e+00 - val_accuracy: 0.6571
Epoch 4/20
44/44 [==============================] - 1s 25ms/step - loss: 0.0000e+00 - accuracy: 0.9449 - val_loss: 0.0000e+00 - val_accuracy: 0.6548
Epoch 5/20
44/44 [==============================] - 1s 23ms/step - loss: 0.0000e+00 - accuracy: 0.9521 - val_loss: 0.0000e+00 - val_accuracy: 0.6571
Epoch 6/20
44/44 [==============================] - 1s 23ms/step - loss: 0.0000e+00 - accuracy: 0.9471 - val_loss: 0.0000e+00 - val_accuracy: 0.6595
Epoch 7/20
44/44 [==============================] - 1s 23ms/step - loss: 0.0000e+00 - accuracy: 0.9536 - v

In [51]:
model.evaluate(wow,check_label)

14/14 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.6571


[0.0, 0.6571428775787354]